In [223]:
import os
import urllib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [224]:
today = datetime.now().strftime('%Y-%m-%d')
yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
month_year = datetime.now().strftime('%b-%Y')
z = r'C:\Users\awvie\Documents'

In [225]:
csv = (r'C:\Users\awvie\JupyterNotebooks\tutorials\scratch' + r'csv.csv')

In [226]:
def QAQC_datapull(a,b,c,d):
    '''
    Sends a data request to the CoAgMET server and processes the data into
    an easy to read dataframe
    
    Arguments
    - a = temporal frequency of the data 
        -must be entered as a string
        -options:
            -'five_minute'
            -'hourly'
            -'daily'
    - b = a five character ID for one or more of the 87 CoAgMET stations
        -must be entered as a string
        -when calling data from multiple stations separate stations IDs
        with a comma
        -Station IDs can be found at https://coagmet.colostate.edu/station_index.php
    - c = first day of the period from which you would like to request data
        -must be entered as a string in 'yyyy-mm-dd' format
    - d = last day of the period from which you would like to request data
        -must be entered as a string in 'yyyy-mm-dd' format
    '''
    # Determines which set of elements to request depending on the data type
    if a == 'hourly':
        e = 'tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir'
        r = pd.date_range(start=c,end=d,freq='H')
    elif a == 'five_minute':
        e = 'tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir'
        r = pd.date_range(start=c,end=d,freq='5min')
    elif a == 'daily':
        e = ('tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,' + 
        'st5mx,st5mn,st15mx,st15mn,gust,gustdir')
        r = pd.date_range(start=c,end=d,freq='D')
        
    try:
        # Makes the data request and loads the data into a csv
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?' +
            'type=' + a +
            '&sids=' + b +
            '&sdate=' + c +
            '&edate=' + d +
            '&elems=' + e,
            filename=csv)
    except Exception as e:
        print('Something went wrong while pulling data from ' + 
              b + ': ' + e.args[0])
    
    
    # Loads the data from the csv into a pandas DataFrame
    data = pd.read_csv(csv)
    data = data.reset_index()

    # Converts the elements string into a list so they can be used 
    # as headers in the dataframe
    headers = e.split(',')
    # Adds to objects to the beginning of list for columns that are added by default
    headers.insert(0,'date')
    headers.insert(0,'station')
    # Assigns the objects in the list as column headers in the dataframe
    data.columns = headers
    
    # Instructs the computer to recognize values in the 'date' column are timestamps
    data['date'] = pd.to_datetime(data.date)
    
    # Creates a datetime index depending on the temporal frequency
    # of the data requested
    if a == 'hourly':
        r = pd.date_range(start=c,end=d,freq='H')
    elif a == 'five_minute':
        r = pd.date_range(start=c,end=d,freq='5min')
    elif a == 'daily':
        r = pd.date_range(start=c,end=d,freq='D')
    
    # Reindexes the CoAgMET data against the newly created datetime index to
    # make it easier to account for missing data
    data = data.set_index('date').reindex(r,copy=False).rename_axis('date')

    # Ensures that all values in the 'station' column are correct
    data['station'] = b
    
    return data

In [227]:
def high_report(df,element,name,f):
    n = 0
    for index,rows in df.iterrows():
        station = str(df['station'].iloc[n])
        datim = str(df['date'].iloc[n])
        off_val = str(df[element].iloc[n])
        return f.write(f'{station} reported an impossibly high {name} at {datim} : {off_val}\n')
        n = n + 1

In [228]:
def low_report(df,element,name,f):
    n = 0
    for index,rows in df.iterrows():
        station = str(df['station'].iloc[n])
        datim = str(df['date'].iloc[n])
        off_val = str(df[element].iloc[n])
        return f.write(f'{station} reported an impossibly low {name} at {datim} : {off_val}\n')
        n = n + 1

In [229]:
def null_report(df,name,f):
    n = 0
    for index,rows in df.iterrows():
        station = str(df['station'].iloc[n])
        datim = str(df['date'].iloc[n])
        return f.write(f'{station} is missing {name} data at {datim}\n')
        n = n + 1

In [230]:
def negative_report(df,name,f):
    n = 0
    for index,rows in df.iterrows():
        station = str(df['station'].iloc[n])
        datim = str(df['date'].iloc[n])
        return f.write(f'{station} reported negative {name} at  {datim}\n')

In [244]:
def nothing_to_report(f,error_type,where):
    if error_type == missing:
        etype = 'missing data'
    elif error_type == negs:
        etype = 'negative values'
    elif error_type == toohigh:
        etype = 'impossibly high values'
    elif error_type == toolow:
        etype = 'impossibly low values'
        
    f.write(f'No {etype} to report of from {where}!\n')

In [232]:
def all_accounted_for(df):
    return len(df) == 0

In [233]:
def missing_all_data(f):
    f.write('Missing all data :(\n')

In [234]:
def create_report(x):
    # Stores a path in which te report will be stored
    y = x['path'] + r'\_' + month_year
    # Creates the pathway if it doesn't already exist
    if not os.path.exists(y): os.makedirs(y)
    # Creates a full name for the file that will be created    
    file = (y + r'\_' + yesterday + '_' + x['drainage'] + 
            '_' + x['region'] + '_QAQC_report.txt')
    return file

In [235]:
def missing_check(df,element):
    nulls = df.loc[pd.isnull(df[element])]
    return nulls

In [236]:
def neg_check(df,element):
    negative = df.loc[df[element] < 0.0]
    return negative

In [237]:
def temp_check(df,x,f):
    site = df['station']
    if x == 'tmean':
        appelle = 'temperature'
    elif x == 'tave':
        appelle = 'daily average temperature'
    elif x == 'tmax':
        appelle = 'daily maximum temperature'
    elif x == 'tmin':
        appelle = 'daily minimum temperature'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        missing = missing_check(df,x)
        toohigh = df.loc[df[x] > 46.1]
        toolow = df.loc[df[x] < 51.6]
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)
        nothing_to_report(f) if all_accounted_for(toohigh) else high_report(toohigh,x,appelle,f)
        nothing_to_report(f) if all_accounted_for(toolow) else low_report(toolow,x,appelle,f)

In [238]:
def rh_check(df,x,f):
    if x == 'rh':
        appelle = 'relative humidity'
    elif x == 'rhmax':
        appelle = 'daily max relative humidity'
    elif x == 'rhmin':
        appelle = 'daily min relative humidity'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        missing = missing_check(df,x)
        toohigh = df.loc[df[x] > 1.0]
        toolow = df.loc[df[x] < 0.01]
        negs = neg_check(df,x)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)
        nothing_to_report(f) if all_accounted_for(toohigh) else high_report(toohigh,x,appelle,f)
        nothing_to_report(f) if all_accounted_for(toolow) else low_report(toolow,x,appelle,f)
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)

In [239]:
def sr_check(df,f):
    appelle = 'solar radiation'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        neg = neg_check(df,'sr')
        missing = missing_check(df,'sr')
        nothing_to_report(f) if all_accounted_for(neg) else negative_report(neg,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)

In [240]:
def ws_check(df,x,f):
    appelle = 'wind speed'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        negs = neg_check(df,'ws')
        missing = missing_check(df,'ws')
        toohigh = df.loc[df['ws'] > 77.785]
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)
        nothing_to_report(f) if all_accounted_for(toohigh) else high_report(toohigh,x,appelle,f)

In [241]:
def vec_check(df,f):
    appelle = 'wind direction'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        negs = neg_check(df,'wind_vec')
        missing = missing_check(df,'wind_vec')
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)

In [242]:
def soiltemp_check(df,x,f):
    if x == 'st5':
        appelle = '5cm soil temp'
    elif x == 'st15':
        appelle = '15cm soil temp'
    elif x == 'st5mx':
        appelle = 'daily maximum 5cm soil temperature'
    elif x == 'st5mn':
        appelle = 'daily minimum 5cm soil temperature'
    elif x == 'st15mx':
        appelle = 'daily maximum 15cm soil temperature'
    elif x == 'st15mn':
        appelle = 'daily minimum 15cm soil temperature'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        toohigh = df.loc[df[x] > 50.0]
        toolow = df.loc[df[x] > -15.0]
        missing = missing_check(df,x)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)
        nothing_to_report(f) if all_accounted_for(toohigh) else high_report(toohigh,x,appelle,f)
        nothing_to_report(f) if all_accounted_for(toolow) else low_report(toolow,x,appelle,f)

In [216]:
def gust_check(df,x,f):
    appelle = 'gust speed'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        negs = neg_check(df,'gust')
        missing = missing_check(df,'gust')
        toohigh = df.loc[df['gust'] > 77.785]
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)
        nothing_to_report(f) if all_accounted_for(toohigh) else high_report(toohigh,x,appelle,f)

In [217]:
def pp_check(df,f):
    appelle = 'precip'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        negs = neg_check(df,'pp')
        missing = missing_check(df,'pp')
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)

In [218]:
def gustdir_check(df,f):
    appelle = 'gust direction'
    if all_accounted_for(df):
        missing_all_data(f)
    else:
        negs = neg_check(df,'gustdir')
        missing = missing_check(df,'gustdir')
        nothing_to_report(f) if all_accounted_for(negs) else negative_report(negs,appelle,f)
        nothing_to_report(f) if all_accounted_for(missing) else null_report(missing,appelle,f)

In [219]:
Collegiate_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Collegiate_Valley'),
    'stations':'bnv01,sld01',
    'drainage':'Arkansas',
    'region':'Collegiate_Valley'
}
East_Valley_Bottom = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\East_Valley_Bottom'),
    'stations':'hly01,hly02,lam01,lam03,lam04,mcl01',
    'drainage':'Arkansas',
    'region':'East_Valley_Bottom'
}
Mesas = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Mesas'),
    'stations':'cnn01,hne01,pnr01,wcf01',
    'drainage':'Arkansas',
    'region':'Mesas'
}
North_of_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\North_of_Valley'),
    'stations':'scm01',
    'drainage':'Arkansas',
    'region':'Mesas'
}
Plateau_South_of_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Plateau_South_of_Valley'),
    'stations':'wls01',
    'drainage':'Arkansas',
    'region':'Plateau_South_of_Valley'
}
West_Valley_Bottom = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\West_Valley_Bottom'),
    'stations':'avn01,fwl01,ljt01,rfd01',
    'drainage':'Arkansas',
    'region':'West_Valley_Bottom'
}
Lower_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Lower_Valley'),
    'stations':'cbl01,slt01',
    'drainage':'Colorado',
    'region':'Lower_Valley'
}
Eagle_River_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Eagle_River_Valley'),
    'stations':'egl01,gyp01',
    'drainage':'Colorado',
    'region':'Eagle_River_Valley'
}
Four_Corners = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Four_Corners'),
    'stations':'ctz01,dvc01,mnc01,twc01,yjk01,yuc01',
    'drainage':'Colorado',
    'region':'Four_Corners'
}
Grand_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Grand_Valley'),
    'stations':'cbn01,frt03,orm02',
    'drainage':'Colorado',
    'region':'Grand_Valley'
}
Headwaters = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Headwaters'),
    'stations':'gby01,krm01,wfd01',
    'drainage':'Colorado',
    'region':'Headwaters'
}
Lower_Gunnison = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Lower_Gunnison'),
    'stations':'dlt01,ekt01,hot02,mtr01,oth01',
    'drainage':'Colorado',
    'region':'Lower_Gunnison'
}
San_Juan = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\San_Juan'),
    'stations':'drg01,ign01,kln01,pgs01',
    'drainage':'Colorado',
    'region':'San Juan'
}
San_Miguel_Paradox = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\San_Miguel_Paradox'),
    'stations':'brk01,nwd01',
    'drainage':'Colorado',
    'region': r'San_Miguel_Paradox'
}
Upper_Gunnison = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Upper_Gunnison'),
    'stations':'gun01',
    'drainage':'Colorado',
    'region':'Upper_Gunnison'
}
Yampa_White = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Yampa_White'),
    'stations':'clk01,hyd01,mkr01',
    'drainage':'Colorado',
    'region': 'Yampa_White'
}
North_Plains = {
    'path':(z + r'\QAQC_reports\Kansas_Drainage\North_Plains'),
    'stations':'akr02,hxt01,hyk02,ilf01,pai01,stg01,wry02,yum02',
    'drainage':'Kansas',
    'region':'North_Plains'
}
South_Plains = {
    'path':(z + r'\QAQC_reports\Kansas_Drainage\South_Plains'),
    'stations':'brl02,brl03,idl01,sbt01,stn01',
    'drainage':'Kansas',
    'region':'South_Plains'
}
Front_Range_Foothills = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\Front_Range_Foothills'),
    'stations':'ckp01',
    'drainage':'Platte',
    'region':'Front_Range_Foothills'
}
Lower_Plains = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\Lower_Plains'),
    'stations':'brg01,hxt01,ksy01,ksy02',
    'drainage':'Platte',
    'region':'Lower_Plains'
}
North_Front_Range = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\North_Front_Range'),
    'stations':'alt01,fcc01,fcl01,ftc01,ftc03,gly04,lcn01',
    'drainage':'Platte',
    'region':'North_Front_Range'
}
North_Park = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\North_Park'),
    'stations':'cow01,heb01,lar01',
    'drainage':'Platte',
    'region':'North_Park'
}
South_Front_Range = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\South_Front_Range'),
    'stations':'ccr01,cht01,eac01,ftl01,lsl01,pkh01',
    'drainage':'Platte',
    'region':'South_Front_Range'
}
South_Park = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\South_Park'),
    'stations':'bnv01,jfn01,sld01',
    'drainage':'Platte',
    'region':'South_Park'
}
San_Luis_Valley = {
    'path':(z + r'\QAQC_reports\Rio_Grande_Drainage\San_Luis_Valley'),
    'stations':'bla01,ctr01,ctr02,ljr01,san01',
    'drainage':'Rio_Grande',
    'region':'San_Luis_Valley'
}


region_list = [Collegiate_Valley,East_Valley_Bottom,Mesas,North_of_Valley,
               Plateau_South_of_Valley,West_Valley_Bottom,Lower_Valley,
               Eagle_River_Valley,Four_Corners,Grand_Valley,Headwaters,
               Lower_Gunnison,San_Juan,San_Miguel_Paradox,Upper_Gunnison,
               Yampa_White,North_Plains,South_Plains,Front_Range_Foothills,
               Lower_Plains,North_Front_Range,North_Park,South_Front_Range,
               South_Park,San_Luis_Valley]

In [220]:
def QC_report_by_region(x):
    '''
    Runs through the previous days data each region of the QC and reports errors 
    into a text file
    '''
    file = create_report(x)
    f = open(file,'w+')
    title = (yesterday + r' QA/QC Report for stations in the ' + x['region'] +
            ' region of the ' + x['drainage'] + ' drainage\n')
    f.write(title + '\n')
    fmdf = pd.DataFrame()
    station_list = x['stations'].split(',')
    f.write('Five Minute Data\n')
    f.write('------------------\n')
    for x in station_list:
        try:
            fmdf = fmdf.append(QAQC_datapull('five_minute',x,yesterday,today))
        except ValueError:
            f.write(x + ' is missing all five minute data\n')
    fmdf = fmdf.reset_index()
    
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
    
    temp_check(fmdf,'tmean',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('------------------------------\n')
    
    rh_check(fmdf,'rh',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    
    sr_check(fmdf,f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Speed Data Report\n')
    f.write('----------------------\n')
    
    ws_check(fmdf,'ws',f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Direction Data Report\n')
    f.write('--------------------------\n')
    
    vec_check(fmdf,f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    
    pp_check(fmdf,f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    soiltemp_check(fmdf,'st5',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    soiltemp_check(fmdf,'st15',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    gust_check(fmdf,'gust',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    gustdir_check(fmdf,f)
    
    f.write('\n')
    f.write('\n')
    
    hdf = pd.DataFrame()
    
    f.write('Hourly Data\n')
    f.write('------------\n')
    f.write('Missing data\n')
    f.write('------------\n')
    
    for x in station_list:
        try:
            hdf = hdf.append(QAQC_datapull('hourly',x,yesterday,today))
        except ValueError:
            f.write(x + ' is missing all hourly data\n')
    
    hdf = hdf.reset_index()

    #missing(hdf,f)

    f.write('\n')
    f.write('\n')
        
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
    
    temp_check(hdf,'tmean',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('-----------------------------\n')
    
    rh_check(hdf,'rh',f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    sr_check(hdf,f)
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Speed Data Report\n')
    f.write('----------------------\n')
    
    ws_check(hdf,'ws',f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Direction Data Report\n')
    f.write('--------------------------\n')
    
    vec_check(hdf,f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    pp_check(hdf,f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    soiltemp_check(hdf,'st5',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    soiltemp_check(hdf,'st15',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    gust_check(hdf,'gust',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    gustdir_check(hdf,f)
    
    f.write('Daily Data\n')
    f.write('----------\n')
    f.write('Missing data:\n')
    f.write('-------------\n')     
    
    dydf = pd.DataFrame()
    
    for x in station_list:
        try:
            dydf = dydf.append(QAQC_datapull('daily',x,yesterday,yesterday))
        except ValueError:
            f.write(x + ' is missing all daily data\n')
    
    dydf = dydf.reset_index()
    
    #missing(dydf,f)
   
    f.write('\n')
    f.write('\n')
        
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
         
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('-----------------------------\n')
    
    rh_check(dydf,'rhmax',f)
    rh_check(dydf,'rhmin',f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    
    sr_check(dydf,f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    
    pp_check(dydf,f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    soiltemp_check(dydf,'st5mx',f)
    soiltemp_check(dydf,'st5mn',f)

    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    soiltemp_check(dydf,'st15mx',f)
    soiltemp_check(dydf,'st5mx',f)
        
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    gust_check(dydf,'gust',f)
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    gustdir_check(dydf,f)
    
    f.close()

In [221]:
for region in region_list:
    QC_report_by_region(region)

In [222]:
test = missing_check(QAQC_datapull('five_minute','bnv01',yesterday,today),'ws')
len(test) == 0

True

In [ ]:
def all_accounted_for(df):
    return len(df) == 0

In [ ]:
nothing_to_report(missing_check(QAQC_datapull('five_minute','bnv01',yesterday,today),'ws'))

In [ ]:
def nothing_to_report(f):
    f.write('Nothing to report!/n')